In [3]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
import time 
import datetime
from datetime import datetime

In [4]:
h2o.init() # start h2o
# 2 cluster 1.759Gb

Checking whether there is an H2O instance running at http://localhost:54321..... not found.


H2OStartupError: Cannot start local server: h2o.jar not found. Paths searched:
    /Users/tim.wu/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
    /Users/tim.wu/anaconda3/h2o_jar/h2o.jar
    /usr/local/h2o_jar/h2o.jar
    /Users/tim.wu/anaconda3/local/h2o_jar/h2o.jar
    /Users/tim.wu/.local/h2o_jar/h2o.jar
    /Users/tim.wu/anaconda3/h2o_jar/h2o.jar


In [3]:
dataPath = "http://coursera.h2o.ai/house_data.3487.csv"
data = h2o.import_file(path=dataPath)  # import dataset to h2oframe

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# generate date feature to year and month
data['year'] = data['date'].substring(0,4)
data['month'] = data['date'].substring(4,6)
# transform some features to categorical features
data['waterfront'] =  data['waterfront'].asfactor()
data['view'] =  data['view'].asfactor()
data['condition'] =  data['condition'].asfactor()
data['grade'] =  data['grade'].asfactor()
data['zipcode'] =  data['zipcode'].asfactor()
data['year'] =  data['year'].asfactor()
data['month'] =  data['month'].asfactor()
# split 90% for train and 10% for test
data_split = data.split_frame(ratios=[0.9],seed=123)  
train = data_split[0]
test = data_split[1]
X = ['bedrooms','bathrooms','sqft_living',
     'sqft_lot','floors','waterfront',
     'view','condition','grade',
     'sqft_above','sqft_basement','yr_built',
     'yr_renovated','zipcode','lat',
     'long','sqft_living15','sqft_lot15']
y = 'price'

In [5]:
# train a linear regression model in 10-fold cross validation with lasso regularization 
# set seed to make sure reproducible
model_v1 = H2OGeneralizedLinearEstimator(family= "gaussian", lambda_= 0.1, alpha = 1,
                                         nfolds = 10, seed = 123,keep_cross_validation_predictions=True)
model_v1.train(X, y, training_frame= train)
print('rmse for penalized regression model:{}'.format(model_v1.rmse(xval=True)))
#h2o.save_model(model=model_v1, path="/mymodel", force=True) # save model

glm Model Build progress: |███████████████████████████████████████████████| 100%
rmse for penalized regression model:151138.85161107054


In [6]:
# train a gbm model in 10-fold cross validation 
# set seed to make sure reproducible
model_v2 = H2OGradientBoostingEstimator(
    ntrees=400,
    learn_rate=0.2,
    max_depth=8,
    stopping_tolerance=0.01, 
    stopping_rounds=2,
    score_each_iteration=True,
    seed=123,
    nfolds = 10,
    keep_cross_validation_predictions=True
)
model_v2.train(X, y, training_frame= train)
print('rmse for GBM model:{}'.format(model_v2.rmse(xval=True)))
#h2o.save_model(model=model_v2, path="/mymodel", force=True) # save model

gbm Model Build progress: |███████████████████████████████████████████████| 100%
rmse for GBM model:122640.99558214223


In [7]:
# train a random forest  model in 10-fold cross validation 
# set seed to make sure reproducible
model_v3 = H2ORandomForestEstimator(
    ntrees=50,
    max_depth=10,
    stopping_tolerance=0.01, 
    stopping_rounds=2,
    score_each_iteration=True,
    seed=123,
    nfolds = 10,
    keep_cross_validation_predictions=True
)
model_v3.train(X, y, training_frame= train)
print('rmse for RF model:{}'.format(model_v3.rmse(xval=True)))
#h2o.save_model(model=model_v3, path="/mymodel", force=True) # save model

drf Model Build progress: |███████████████████████████████████████████████| 100%
rmse for RF model:137898.55625461036


In [8]:
# train a deep learning model in 10-fold cross validation 
# set seed to make sure reproducible
model_v4 = H2ODeepLearningEstimator(epochs=10,
                                    nfolds=10,
                                    seed=123,
                                    l1=1e-5,
                                    l2=1e-5,
                                    activation="RectifierWithDropout",
                                    hidden=[256,512,64],
                                    input_dropout_ratio = 0.1,
                                    hidden_dropout_ratios = [0.15,0.2,0.05],
                                    keep_cross_validation_predictions=True)
model_v4.train(X, y, training_frame= train)
print('rmse for DL model:{}'.format(model_v4.rmse(xval=True)))
#h2o.save_model(model=model_v4, path="/mymodel", force=True) # save model

deeplearning Model Build progress: |██████████████████████████████████████| 100%
rmse for DL model:126786.24420521386


In [9]:
# train a ensemble model based on four models above
# set seed to make sure reproducible
ensemble = H2OStackedEnsembleEstimator(base_models=[model_v4,model_v3, model_v2,model_v1],
                                       metalearner_algorithm = 'deeplearning',seed=123, 
                                       metalearner_params = {'hidden':[128,384,512],'activation':"RectifierWithDropout",
                                                             'input_dropout_ratio': 0.1}
                                       )
ensemble.train(X, y, training_frame=train)
print('rmse for ensemble model:{}'.format(ensemble.rmse()))
#h2o.save_model(model=ensemble, path="/mymodel", force=True) # save model

stackedensemble Model Build progress: |███████████████████████████████████| 100%
rmse for ensemble model:101659.27596982196


In [10]:
# get the model performance on test set
ensemble.model_performance(test)

## the RMSE is 121,523 which is an RMSE below 123,000.


ModelMetricsRegression: stackedensemble
** Reported on test data. **

MSE: 15337386361.086788
RMSE: 123844.2019679839
MAE: 71817.28892240314
RMSLE: 0.17790808317400533
Mean Residual Deviance: 15337386361.086788


In [11]:
h2o.cluster().shutdown()

H2O session _sid_81e8 closed.
